In [1]:
import gdspy
import numpy as np

In [2]:
gdsii = gdspy.GdsLibrary()
file = '/Users/wendy/Desktop/Wendy-qiskit-code/export_test_0506_LL.gds'
gdsii.read_gds(file, units = 'import')

gds_new = gdspy.GdsLibrary(unit = 1e-6, precision = 1e-12)
# gds_new.cells.clear()

In [3]:
top = gds_new.new_cell('TOP')



In [4]:
qubit_layer = 5
junction_layer = 20
ab_layer = 31
ab_square_layer = 30
junction_area_layer = 60
chip_edge_layer = int(703)

layer_numbers  = [qubit_layer,junction_layer,ab_layer,ab_square_layer]#,junction_area_layer]

In [5]:
qubit_layer_name = 'TOP_M1'
junction_name = 'TOP_SE1'
ab_name = 'TOP_BR'
ab_square_name = 'TOP_RR'
junction_area_name = 'TOP_SE1_JJ'
chip_edge_name = 'TOP_ChipEdge'



In [6]:
qubit_cell = gdspy.Cell(qubit_layer_name)
junction_cell = gdspy.Cell(junction_name)
ab_cell = gdspy.Cell(ab_name)
ab_square_cell = gdspy.Cell(ab_square_name)
junction_area_cell = gdspy.Cell(junction_area_name)
chip_edge_cell = gdspy.Cell(chip_edge_name)


In [7]:
cell_names = [qubit_cell, junction_cell, ab_cell,ab_square_cell]#, junction_area_cell]

In [8]:
data_type = 0

In [9]:
def snap_to_grid(coord, grid_size):
    return np.round(coord / grid_size) * grid_size

In [10]:
qubit_layer_grid = 50*1e-3
junction_layer_grid = 5*1e-3

In [11]:
polys = gdsii.top_level()[0].get_polygons(by_spec = True)

In [12]:
keys = list(polys.keys())

In [13]:
keys

[(20, 10), (5, 10), (5, 11), (5, 0), (30, 10), (31, 10)]

In [14]:
datatypes = {}
for i in keys:
    try:
        print(datatypes[i[0]])
    except:
        datatypes[i[0]] = [i[1]]
    else:
        datatypes[i[0]].append(i[1])


[10]
[10, 11]


In [15]:
datatypes

{20: [10], 5: [10, 11, 0], 30: [10], 31: [10]}

In [16]:
chip_edge_layer

703

In [17]:
for i, layer in enumerate(layer_numbers):
    for j in datatypes[layer]:
        source_polyons = polys.get((layer,j),[])

        for polygon in source_polyons:
            p =gdspy.Polygon(polygon, layer = int(layer), datatype = 0)
            # print(cell_names[i])
            top.add(p)

In [18]:
junction_area_cell.add(gdspy.Polygon(polygon[:-1], layer = int(junction_area_layer)))

In [19]:
p = gdspy.Polygon(polygon[:-1], layer = 0)
chip_edge_cell.add(p)

In [20]:
p

In [21]:
# rectangle = gdspy.Polygon(np.array([[-500.5,500.5],[500.5,500.5],[500.5,-500.5],[-500.5,-500.5]]), layer = int(chip_edge_layer))
# chip_edge_cell.add(rectangle)

In [22]:
gds_new.write_gds('compatible_try2.gds')

In [23]:
# ref1 = gdspy.CellReference(qubit_cell)
# ref2 = gdspy.CellReference(junction_cell)
# ref3 = gdspy.CellReference(ab_cell)
# ref4 = gdspy.CellReference(ab_square_cell)
# ref5 = gdspy.CellReference(junction_area_cell)
# ref6 = gdspy.CellReference(chip_edge_cell)

# top.add(ref1)
# top.add(ref2)
# top.add(ref3)
# top.add(ref4)
# top.add(ref5)
# top.add(ref6)

In [24]:
qubit_cell.get_polygons(by_spec = True)

{}

In [25]:
# top = qubit_cell = gds_new.new_cell('TOP')
# rec = gdspy.Polygon(np.array([[-500.,500.],[500.,500.],[500.,-500.],[-500.,-500.]]), layer = int(0))
# top.add(rec)

In [26]:
gds_new.write_gds('compatible_try21.gds')

In [27]:
gds_new.top_level()[0].get_polygons()

[array([[-3.39335,  4.8    ],
        [-3.3    ,  4.85   ],
        [-3.3    ,  4.95   ],
        [-3.5    ,  4.95   ],
        [-3.5    ,  4.85   ],
        [-3.40665,  4.8    ],
        [-3.40665,  4.792  ],
        [-3.39335,  4.792  ],
        [-3.39335,  4.8    ]]),
 array([[-4.8    ,  3.40665],
        [-4.85   ,  3.5    ],
        [-4.95   ,  3.5    ],
        [-4.95   ,  3.3    ],
        [-4.85   ,  3.3    ],
        [-4.8    ,  3.39335],
        [-4.792  ,  3.39335],
        [-4.792  ,  3.40665],
        [-4.8    ,  3.40665]]),
 array([[-4.8    ,  2.55665],
        [-4.85   ,  2.65   ],
        [-4.95   ,  2.65   ],
        [-4.95   ,  2.45   ],
        [-4.85   ,  2.45   ],
        [-4.8    ,  2.54335],
        [-4.792  ,  2.54335],
        [-4.792  ,  2.55665],
        [-4.8    ,  2.55665]]),
 array([[-4.8    ,  1.70665],
        [-4.85   ,  1.8    ],
        [-4.95   ,  1.8    ],
        [-4.95   ,  1.6    ],
        [-4.85   ,  1.6    ],
        [-4.8    ,  1.69335],
    

In [28]:
gds_new.cells

{'TOP': <gdspy.library.Cell at 0x145a58dc0>}

In [29]:
gdsii.top_level()

In [30]:
polygon

array([[ 4.797  , -3.37526],
       [ 4.797  , -3.38526],
       [ 4.787  , -3.38526],
       [ 4.787  , -3.37526],
       [ 4.797  , -3.37526]])